In [ ]:
#hide
from fastai.vision.all import *
from ipywidgets import widgets 
from fastai.vision.widgets import *
from my_timesaver_utils.bayes_inference import *

In [ ]:
#hide
path = Path()
# path.ls(file_exts='.pkl')

In [ ]:
learn_inf = load_learner(path/'export.pkl')

# My Very Own (Really Cool!)  Image Classifier with a Fail (Uncertainty) Detector

Classifier Categories:

In [ ]:
for category in learn_inf.dls.vocab:
    print(category)

In [ ]:
def singularize(word):
    prefix = 'a '
    if word[-1] == 's':
        word = word[:-1]
    if word[0] in ('a','e','i','o','u'):
        prefix = 'an '
    return prefix + word

def handle_bayes_prediction_with_uncertainty(res):
    pass_threshold = res[0]
    best_cat = singularize(res[7])
    best_prob = res[5] * 100.
    best_unc = res[4]
    unc_msg1 = f"Doesn't look like anything to me, but it could be {best_cat}"
    unc_msg2 = f' with a probability of {best_prob:.2f} percent ' 
    unc_msg3 = f'and uncertainty(entropy) of {best_unc:.2f}'
    uncertain_msg = unc_msg1 + unc_msg2 + unc_msg3
    certain_msg = f'To me, that looks like {best_cat}, with a probability of {best_prob:.2f}'
    if not pass_threshold: 
        return uncertain_msg
    return certain_msg

btn_upload = SimpleNamespace(data = ['images/purple_dog.jpg'])
img = PILImage.create(btn_upload.data[-1])
out_pl = widgets.Output()
out_pl.clear_output()
with out_pl: display(img.to_thumb(200,200))
res = learn_inf.bayes_predict_with_uncertainty(img)

lbl_pred = widgets.Label()
lbl_pred.value = handle_bayes_prediction_with_uncertainty(res)
btn_run = widgets.Button(description='Classify')

def on_click_classify(change):
    img = PILImage.create(btn_upload.data[-1])
    out_pl.clear_output()
    with out_pl: display(img.to_thumb(200,200))
    res = learn_inf.bayes_predict_with_uncertainty(img)
    lbl_pred.value = handle_bayes_prediction_with_uncertainty(res)

btn_run.on_click(on_click_classify)
#Putting back btn_upload to a widget for next cell
btn_upload = widgets.FileUpload()
#hide_output
VBox([widgets.Label('UPLOAD AN IMAGE AND CLASSIFY!'), 
      btn_upload, btn_run, out_pl, lbl_pred])